In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
df = pd.read_csv("/content/train (1).csv",encoding='ISO-8859-1')
df.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
df = df.drop(['id','keyword','location'],axis=1)
df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
df_0_class = df[df['target']==0]
df_1_class = df[df['target']==1]
df_0_class_undersampled = df_0_class.sample(df_1_class.shape[0])
df = pd.concat([df_0_class_undersampled, df_1_class], axis=0)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['target'], stratify=df['target'])

In [ ]:
!pip install tensorflow-text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 61.5 MB/s eta 0:00:00


In [ ]:
preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text-layer')
preprocessed_text = preprocess(text_input)
outputs = encoder(preprocessed_text)
d_layer = tf.keras.layers.Dropout(0.1, name="dropout-layer")(outputs['pooled_output'])
d_layer = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(d_layer)
model = tf.keras.Model(inputs=[text_input], outputs = [d_layer])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text-layer (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text-layer[0][0]']             
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
m= [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=m)

In [ ]:
model.fit(X_train, y_train, epochs=10)
model.evaluate(X_test, y_test)

Epoch 1/10
154/154 [==============================] - 1946s 13s/step - loss: 0.6424 - accuracy: 0.6298 - precision: 0.6236 - recall: 0.6551
Epoch 2/10
154/154 [==============================] - 1907s 12s/step - loss: 0.6085 - accuracy: 0.6863 - precision: 0.6847 - recall: 0.6906
Epoch 3/10
154/154 [==============================] - 1939s 13s/step - loss: 0.5958 - accuracy: 0.6910 - precision: 0.6888 - recall: 0.6967
Epoch 4/10
154/154 [==============================] - 1933s 13s/step - loss: 0.5796 - accuracy: 0.7051 - precision: 0.7051 - recall: 0.7049
Epoch 5/10
154/154 [==============================] - 1973s 13s/step - loss: 0.5759 - accuracy: 0.7077 - precision: 0.7098 - recall: 0.7028
Epoch 6/10
154/154 [==============================] - 1946s 13s/step - loss: 0.5663 - accuracy: 0.7159 - precision: 0.7166 - recall: 0.7142
Epoch 7/10
154/154 [==============================] - 1891s 12s/step - loss: 0.5687 - accuracy: 0.7144 - precision: 0.7186 - recall: 0.7049
Epoch 8/10
154/154 [

[0.5400314331054688, 0.7322738170623779, 0.7567567825317383, 0.684596598148346]

In [ ]:
import numpy as np
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
y_predicted = np.where(y_predicted > 0.5, 1, 0)
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_predicted)
matrix

52/52 [==============================] - 650s 12s/step


array([[638, 180],
       [258, 560]])

In [ ]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.71      0.78      0.74       818
           1       0.76      0.68      0.72       818

    accuracy                           0.73      1636
   macro avg       0.73      0.73      0.73      1636
weighted avg       0.73      0.73      0.73      1636

